In [ ]:
%pip -qqq install -r requirements.txt

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_SESSION_TOKEN']=''

# Use Sagemaker Endpoint for Llama-2 as LLM model in Langchain

In [ ]:
import json
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

def init_llm_sm_endpoint():

    endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-13b-f'
    aws_region='us-east-1'
    parameters = {"max_new_tokens": 700, "temperature": 0.1}


    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        # def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        #     input_str = json.dumps({"inputs": prompt, **model_kwargs})
        #     return input_str.encode("utf-8")
        def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
            input_str = json.dumps({"inputs" : [[{"role" : "system",
            "content" : "You are a kind robot."},
            {"role" : "user", "content" : prompt}]],
            "parameters" : {**model_kwargs}})
            return input_str.encode('utf-8')

        def transform_output(self, output: bytes) -> str:
            response_json = json.loads(output.read().decode("utf-8"))
            return response_json[0]["generation"]["content"]

    content_handler = ContentHandler()

    sm_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name=aws_region,
        model_kwargs=parameters,
        content_handler=content_handler,
        endpoint_kwargs={"CustomAttributes": "accept_eula=true"},
    )
    return sm_llm

llm = init_llm_sm_endpoint()
text = "What would be a good AWS new service name that allow customers to chat with their own data? Keep answer short. Just give me one answer in your reponse"
llm(text)